In [ ]:
# to load model
!pip install -q transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments,Trainer
from peft import get_peft_model,LoraConfig,TaskType
from datasets import Dataset

In [ ]:
data={
    "text":[
        "### Question: What is CNN?\n### Answer:CNN means Convolutional Neural Network.",
        "### Question: What is RNN?\n### Answer:RNN means Recurrent Neural Network."
        "### Question: what is LSTM?\n### Answer:LSTM means Long Short Term Memory and it is alos a type of RNN."
    ]
}

dataset=Dataset.from_dict(data)

In [ ]:
model_name="tiiuae/falcon-rw-1b"
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
def tokenize(batch):
  tokens=tokenizer(
      batch["text"],
      padding="max_length",
      truncation=True,
      max_length=128 #if more than 128 it will truncate
  )
  tokens["labels"]=tokens["input_ids"].copy() #sequence (before word)
  return tokens

In [ ]:
#fitiing all the thing from the above
tokenized_dataset=dataset.map(tokenize,batched=True)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
#finetuning
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto" #hadware config and memory
    )

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [ ]:
#lora configuration for fine tuning
lora_config=LoraConfig(
    r=8, #no of adapters
    lora_alpha=16, #scaling
    target_modules=["query_key_value"], #only here lora works(query_key_value-top 1 important in attention mechanism and trains it)
    lora_dropout=0.1,  #(While working all chances for overfit-so drop here)
    bias="none",
    task_type=TaskType.CAUSAL_LM #causual language modeling
)

In [ ]:
model=get_peft_model(model,lora_config)
model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.1198


In [ ]:
#setting training parameters
training_args=TrainingArguments(
    output_dir=".results/",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    learning_rate=2e-4,
    logging_steps=1,
    save_strategy="no",
    report_to="none"
)

In [ ]:
#train using trainer
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

/tmp/ipython-input-25-3512634422.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,6.279300
2,4.465500
3,4.936600
4,3.889700
5,4.106800
6,3.293300


TrainOutput(global_step=6, training_loss=4.4951984484990435, metrics={'train_runtime': 1.2065, 'train_samples_per_second': 4.973, 'train_steps_per_second': 4.973, 'total_flos': 5576488648704.0, 'train_loss': 4.4951984484990435, 'epoch': 3.0})

In [ ]:
model.save_pretrained("lora-finetuned")
tokenizer.save_pretrained("lora_finetuned")

('lora_finetuned/tokenizer_config.json',
 'lora_finetuned/special_tokens_map.json',
 'lora_finetuned/vocab.json',
 'lora_finetuned/merges.txt',
 'lora_finetuned/added_tokens.json',
 'lora_finetuned/tokenizer.json')